In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 18.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=0ca2039071a7eb9eb88df4439f0834559b76e77334d83beff61925bc4e1cf455
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [65]:
from bs4 import BeautifulSoup
import sys
import os.path
import string
import os
import re
import random
import time
import binascii
import math
import numpy as np
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import collections
import time
import itertools
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


# CÁC HÀM HỖ TRỢ THUẬT TOÁN

In [69]:
def split_k_gram(words):
  # keep word shingles
  shinglesInDocWords = set()

  # keep hashed shingles
  shinglesInDocInts = set()

  shingle = []
  for index in range(len(words) - k_gram + 1):
    # Construct the shingle text by combining k words together.
    shingle = words[index:index + k_gram]
    shingle = ' '.join(shingle)

    # Hash the shingle to a 32-bit integer.
    crc = binascii.crc32(shingle.encode())

    if shingle not in shinglesInDocWords:
      shinglesInDocWords.add(shingle)

    if crc not in shinglesInDocInts:
      shinglesInDocInts.add(crc)
    else:
      del shingle
      index = index - 1
  return shinglesInDocInts

def isPrime(n):  
  # Corner cases
  if(n <= 1):
    return False
  if(n <= 3):
    return True
    
  # This is checked so that we can skip
  # middle five numbers in below loop
  if(n % 2 == 0 or n % 3 == 0):
    return False
    
  for i in range(5,int(math.sqrt(n) + 1), 6):
    if(n % i == 0 or n % (i + 2) == 0):
      return False
  return True
 
# Function to return the smallest
# prime number greater than N
def nextPrimefunc(N):
  # Base case
  if (N <= 1):
    return 2
  prime = N
  found = False
  while(not found):
    prime = prime + 1

    if(isPrime(prime) == True):
      found = True
  return prime

def pickRandomCoeffs(k):
  randList = []

  while k > 0:
    randIndex = random.randint(0, maxShingleID)
    while randIndex in randList:
      randIndex = random.randint(0, maxShingleID)
    randList.append(randIndex)
    k = k - 1

  return randList

def signatureFunction(shingleIDSet):
  signature = []
  for i in range(0, numHashesFunction):
    minHashCode = nextPrime + 1
    for shingleID in shingleIDSet:
      # Evaluate the hash function.
      hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime

      if hashCode < minHashCode:
        minHashCode = hashCode
    signature.append(minHashCode)
  return signature

def FindSimilaryty_Singles(doc, n_neighbor):
  doc = re.sub("[^\w]", " ", doc).split()
  singlesInput = split_k_gram(doc)
  neighbors_of_given_document_Test ={}
  for j in range(len(docsAsShingleSets)):
    singles2 = docsAsShingleSets[j]
    s1 = set(singlesInput)
    s2 = set(singles2)
    J = len(s1.intersection(s2)) / float(len(s1.union(s2)))
    if (float(J) > 0):
        percJ = J * 100
        neighbors_of_given_document_Test[j] = percJ
  neighbors_of_given_documentSIGNATURES_Test= sorted(neighbors_of_given_document_Test.items(), key=lambda x: x[1], reverse=True)
  return list(neighbors_of_given_documentSIGNATURES_Test)[:n_neighbor]

def FindSimilaryty(doc, n_neighbor):
  doc = re.sub("[^\w]", " ", doc).split()
  doc = split_k_gram(doc)
  signatureInput =signatureFunction(doc)
  neighbors_of_given_documentSIGNATURES_Test ={}
  # neighbors_of_given_documentSIGNATURES_Test = signaturesRDD.map(lambda x: Calsimilarity_Hash(x,signatureInput)).collect()
  for j in range(len(signatures)):
    signature2 = signatures[j]
    s1 = set(signatureInput)
    s2 = set(signature2)
    J = len(s1.intersection(s2)) / float(len(s1.union(s2)))
    if (float(J) > 0):
        percJ = J * 100
        neighbors_of_given_documentSIGNATURES_Test[j] = percJ
  # neighbors_of_given_documentSIGNATURES_Test =set(filter(None, neighbors_of_given_documentSIGNATURES_Test))
  neighbors_of_given_documentSIGNATURES_Test= sorted(neighbors_of_given_documentSIGNATURES_Test.items(), key=lambda x: x[1], reverse=True)
  return list(neighbors_of_given_documentSIGNATURES_Test)[:n_neighbor]

def get_band_hashes(np_signature, numBand):
  np_signature  = np.array_split(np_signature, numBand)
  hash_sig = []
  for i in np_signature:
    j = [hash(x) for x in i]
    hash_sig.append(sum(j))
  return hash_sig

def LSH(Test_input,n_neighbor):
  Test_input = re.sub("[^\w]", " ", Test_input).split()
  Test_input = split_k_gram(Test_input)
  signatureInput =signatureFunction(Test_input)
  input_band = get_band_hashes(signatureInput, numBand)
  idDoc_samebucket = []
  result=[]
  for i in range(len(input_band)):
    if input_band[i] in bucket_value :
      id = bucket_value.index(input_band[i])
      idDoc_samebucket += [x for x in bucket_index[id] if x not in idDoc_samebucket]
  for i in idDoc_samebucket:
    s1 = set(input_band)
    s2 = set(get_band_hashes(signatures[i], numBand))
    sim = len(s1.intersection(s2)) / float(len(s1.union(s2))) *100
    result.append((sim, i))
  result= sorted(result, key=lambda x: x[0], reverse=True)
  return list(result)[:n_neighbor]

# SHINGLING - TÌM MA TRẬN BOOLEANS VỚI K-GRAMS = 10


In [70]:
#read data

f = open("/content/drive/MyDrive/BIGDATA/BTL/data3.txt", "r", encoding="utf-8")

data = f.read().split("\n")

conf = SparkConf().setMaster("local").setAppName("BigDataLSH")
sc = SparkContext.getOrCreate(conf=conf)

rdd = sc.parallelize(data)
d =rdd.map(lambda x: re.sub("[^\w]", " ", x).split())

timeShingling = time.time()

k_gram = 10
numHashesFunction = 50

docsAsShingleSetsRDD = d.map(lambda x : split_k_gram(x))
#tong so Singles
shingleNo = sum(docsAsShingleSetsRDD.map(lambda x: len(x)).collect())
#Singles khac nhau
# len(list(set().union(*docsAsShingleSets1.collect())))
print("Thời gian chuyển đổi Shingling:" , time.time() - timeShingling)

Thời gian chuyển đổi Shingling: 6.756944894790649


# MIN-HASHING - CREATE SIGNATURE MATRIX VỚI SỐ HASH FUNCTION = 50


In [71]:
docsAsShingleSets = docsAsShingleSetsRDD.collect()
# print(shingleNo)

In [72]:
maxShingleID = shingleNo
nextPrime = nextPrimefunc(shingleNo)
print ('Next prime = ', nextPrime)

coeffA = pickRandomCoeffs(numHashesFunction)
coeffB = pickRandomCoeffs(numHashesFunction)

Next prime =  3156259


In [73]:
timeSig = time.time()
docNames = [i for i in range(len(docsAsShingleSets))]
signatures = docsAsShingleSetsRDD.map(lambda x: signatureFunction(x)).collect()
numDocs = len(docsAsShingleSets)
print("Thời gian chuyển đổi Signature MinHashing:" , time.time() - timeSig)

Thời gian chuyển đổi Signature MinHashing: 53.45552325248718


# MIN-HASHING - CALCULATE JACCARD SIMILARITY

## SO SÁNH TỐC ĐỘ TÍNH JACCARD SIMILARITY GIỮA SHINGLES VÀ SIGNATURES

In [76]:
inputNewDoc= data[0]
print(data[0])
time_JS = time.time()
resSimilarity = FindSimilaryty_Singles(inputNewDoc, 5)
for x,y in resSimilarity:
  print("{}% index-document:{} | document:{}".format(round(y,2),x, data[x]))
print(time.time() - time_JS)

"Chúng tôi xác nhận rằng nhiều phát đạn đã bắn vào Trung tâm Mỹ ở Yangon hôm nay. Không có ai bị thương, chúng tôi đang điều tra sự việc", phát ngôn viên đại sứ quán Mỹ tại Myanmar Aryani Manring cho biết.Giới chức Yangon, thành phố lớn nhất Myanmar, chưa bình luận về thông tin.Các tổ chức giám sát nhân quyền ở Myanmar cho biết có ít nhất 16 người biểu tình đã chết trong ngày 27/3, nâng tổng số người thiệt mạng kể từ khi quân đội Myanmar tiếp quản quyền lực ngày 1/2 lên 328.Những người biểu tình đã xuống đường gần như mỗi ngày sau khi quân đội bắt Cố vấn Suu Kyi và các quan chức cấp cao trong chính quyền dân cử, với lý do cáo buộc gian lận bầu cử hồi tháng 11/2020 không được giải quyết. Quân đội cam kết chuyển giao quyền lực cho bên chiến thắng sau khi bầu cử được tổ chức lại.Mỹ và các nước phương Tây đang gia tăng áp lực lên chính quyền quân sự Myanmar. Bộ Tài chính Mỹ hôm 25/3 thông báo đưa hai tập đoàn Myanmar Economic Holdings Public Company (MEHL) và Myanmar Economic Corporation L

In [77]:
#test inputNewDoc is a doc
inputNewDoc= data[0]
print(data[0])
time_JShash = time.time()
resSimilarity = FindSimilaryty(inputNewDoc, 5)
print(resSimilarity)
for x,y in resSimilarity:
  print("{}% index-document:{} | document:{}".format(round(y,2),x, data[x]))
print("Time to find by JS_minHash ", time.time() - time_JShash)

"Chúng tôi xác nhận rằng nhiều phát đạn đã bắn vào Trung tâm Mỹ ở Yangon hôm nay. Không có ai bị thương, chúng tôi đang điều tra sự việc", phát ngôn viên đại sứ quán Mỹ tại Myanmar Aryani Manring cho biết.Giới chức Yangon, thành phố lớn nhất Myanmar, chưa bình luận về thông tin.Các tổ chức giám sát nhân quyền ở Myanmar cho biết có ít nhất 16 người biểu tình đã chết trong ngày 27/3, nâng tổng số người thiệt mạng kể từ khi quân đội Myanmar tiếp quản quyền lực ngày 1/2 lên 328.Những người biểu tình đã xuống đường gần như mỗi ngày sau khi quân đội bắt Cố vấn Suu Kyi và các quan chức cấp cao trong chính quyền dân cử, với lý do cáo buộc gian lận bầu cử hồi tháng 11/2020 không được giải quyết. Quân đội cam kết chuyển giao quyền lực cho bên chiến thắng sau khi bầu cử được tổ chức lại.Mỹ và các nước phương Tây đang gia tăng áp lực lên chính quyền quân sự Myanmar. Bộ Tài chính Mỹ hôm 25/3 thông báo đưa hai tập đoàn Myanmar Economic Holdings Public Company (MEHL) và Myanmar Economic Corporation L

# LOCALLITY SENSITIVE HASHING

In [78]:
time_LSH = time.time()

numBand = 50
bucket_value = []
bucket_index = []
# test = [signatures[0], signatures[0], signatures[0],signatures[1]]

for k in range(len(signatures)):
  for i in get_band_hashes(signatures[k], numBand):
    if i not in bucket_value:
      bucket_value.append(i)
      bucket_index.append([k])
    else:
      index = bucket_value.index(i)
      bucket_index[index].append(k)

Test_input = data[1]
lst_result = LSH(Test_input, 5)
for x,y in lst_result:
  print("{}% index-document:{} | document: {}".format(round(x,2),y, data[y]))

print("Thời gian tìm kiểm LSH ", time.time() - time_LSH)

100.0% index-document:1 | document: Người đẹp mặc năng động với crop-top trắng, chân váy kẻ ngang, tạo cảm giác vòng ba đầy đặn hơn.Người đẹp mặc năng động với crop-top trắng, chân váy kẻ ngang, tạo cảm giác vòng ba đầy đặn hơn.Bà mẹ một con giữ dáng bằng cách tập yoga, ăn nhiều hoa quả, rau xanh.Ảnh: Ngô Thanh Sơn. Trang điểm: Hiwon. Stylist: Phạm Bảo Luận.Bà mẹ một con giữ dáng bằng cách tập yoga, ăn nhiều hoa quả, rau xanh.Ảnh: Ngô Thanh Sơn. Trang điểm: Hiwon. Stylist: Phạm Bảo Luận.
100.0% index-document:182 | document: Người đẹp mặc năng động với crop-top trắng, chân váy kẻ ngang, tạo cảm giác vòng ba đầy đặn hơn.Người đẹp mặc năng động với crop-top trắng, chân váy kẻ ngang, tạo cảm giác vòng ba đầy đặn hơn.Bà mẹ một con giữ dáng bằng cách tập yoga, ăn nhiều hoa quả, rau xanh.Ảnh: Ngô Thanh Sơn. Trang điểm: Hiwon. Stylist: Phạm Bảo Luận.Bà mẹ một con giữ dáng bằng cách tập yoga, ăn nhiều hoa quả, rau xanh.Ảnh: Ngô Thanh Sơn. Trang điểm: Hiwon. Stylist: Phạm Bảo Luận.
2.04% index-